# <b> Deep Face Detection Model


### Installing Dependencies

In [ ]:
%pip install matplotlib albumentations --q
%pip install tensorflow-macos --q
%pip install labelme --q

#List of Dependencies that are installed
%pip list


In [ ]:
import tensorflow as tf 
import os
import time
import uuid #Unique File Identifiers for the Images
import cv2
import json 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import shutil
import albumentations as alb

### Capture Images used for Data Segement and Augmentation

In [4]:
#Image Paths 
image_path = os.path.join('/Applications/Deep Learning/Face Detection/data', 'images')
number_of_images = 30

#If Directory Does not Exist
os.makedirs(image_path, exist_ok=True)

In [ ]:
#Open video source 
cap = cv2.VideoCapture(0)

while True:
    answer = input("Do you wish to capture Images(Y/N): ").lower()
    if answer != 'y':
        break
    for image_number in range(number_of_images):
        print('Collecting Image {}'.format(image_number))
        return_val, frame = cap.read() #Frame Number and Boolean Val for Success

        if not return_val:
            print('Image did not Successfully Capture')

        else:
            print('Picture Captured Successfully')
        img_name = os.path.join(image_path, f'{str(uuid.uuid1())}.jpg') #Unique UUID per image
        cv2.imwrite(img_name, frame) #Save Image
        cv2.imshow('frame', frame) #Display Image in Frame
        time.sleep(0.5)

        #Break out of Capturing Images if error occurs
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
   

cap.release()
cv2.destroyAllWindows()



### Annotate DataSet and Build Image Loading Function

In [4]:
#Create a Virtual Enviornment Just for LabelMe
''' 
conda create -n labelme python=3.10
conda activate labelme
pip install labelme
labelme
conda activate labelme (Use to Label the Images)
'''

zsh:1: command not found: labelme


In [8]:
#GPU Avaliable to be used?
print('GPU Avaliable:', tf.test.is_gpu_available('GPU'))

''' 
Load Images into TF Data Pipeline
Create Tensorflow of datasets, shuffle enables not inherent order)
'''
images = tf.data.Dataset.list_files('/Applications/Deep Learning/Face Detection/data/images/*.jpg', shuffle=False)


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Avaliable: False


In [3]:
''' 
Reads entire filestream of images and decodes JPEG file into 3D Tensor
Converts Images into Useable Tensor Datatype
'''
def load_image(img):
    byte_img = tf.io.read_file(img)
    image = tf.io.decode_jpeg(byte_img)
    return image


In [ ]:
#Map function that applies function to each item
images = images.map(load_image)

#Iterate through the images
images.as_numpy_iterator().next()

In [ ]:
#View Raw Images in Matplotlib
image_generator = images.batch(10).as_numpy_iterator()
print(type(image_generator))
next_batch = next(image_generator)


In [ ]:
#.next() contains the index and the image using .as_numpy_iterator()
while True:
    answer = input('Next Batch: ').lower()
    if answer != 'y':
        break
    else:
        plt.clf()
        next_batch = next(image_generator)
    
        fig, ax = plt.subplots(ncols=5, nrows=2, figsize=(10,10))
        for idx, image in enumerate(next_batch):
            row = idx//5
            col = idx%5
            ax[row,col].imshow(image)
            ax[row,col].axis('off')
        plt.show()
     

### Partition Unaugmented Data

* Note: It is possible to do the following task using split_folders (more standard in DL ). Higher level command and ensures that on any OS that the command works correctly. In addition splitting between the Data and the corresponding Label Class is also automatic

In [12]:
#Total Images is 382
print('Total Images in Dataset:', len([f for f in os.listdir(image_path)]))


total_images = [f for f in os.listdir(image_path)]

#Train Dataset should be 70% of dataset
train_set, temp_set = train_test_split(total_images, test_size=0.3, random_state = 42)

#Validation and Test will be 15% each
validation_set, test_set = train_test_split(temp_set, test_size=0.5, random_state = 42)

print('Train: ', len(train_set))
print("Validation:", len(validation_set))
print("Test:", len(test_set))


Total Images in Dataset: 382
Train:  267
Validation: 57
Test: 58


In [13]:
image_path = '/Applications/Deep Learning/Face Detection/data/images'
base_path = '/Applications/Deep Learning/Face Detection/data'


#Put each Train Test Set into Respective Folder
train_dataset = os.path.join(base_path, 'train_set/images')
validation_dataset = os.path.join(base_path, 'validation_set/images')
test_dataset = os.path.join(base_path, 'test_set/images')


#Put Images in Respective Folder
for img in train_set:
     shutil.copy(os.path.join(image_path, img), os.path.join(train_dataset, img))
for img in validation_set:
     shutil.copy(os.path.join(image_path, img), os.path.join(validation_dataset, img))

for img in test_set:
     shutil.copy(os.path.join(image_path, img), os.path.join(test_dataset, img))

#Create Tensorflow dataset (Images)
train_dataset = tf.data.Dataset.list_files(os.path.join(train_dataset, '*.jpg'))
validation_dataset   = tf.data.Dataset.list_files(os.path.join(validation_dataset, '*.jpg'))
test_dataset  = tf.data.Dataset.list_files(os.path.join(test_dataset, '*.jpg'))


In [234]:
#This is not an efficient solution but it does the trick in aligning corresponding label to img
label_path = '/Applications/Deep Learning/Face Detection/data/labels'
train_labels = ["00717294-eb18-11f0-93ba-c64476ca29a9", "00c7b1fe-eb18-11f0-93ba-c64476ca29a9", "011e76f6-eb18-11f0-93ba-c64476ca29a9", "0174d6f4-eb18-11f0-93ba-c64476ca29a9", "2cd419dc-eb17-11f0-93ba-c64476ca29a9", "2d287324-eb17-11f0-93ba-c64476ca29a9", "2d8081fe-eb17-11f0-93ba-c64476ca29a9", "2e2bf7b4-eb17-11f0-93ba-c64476ca29a9", "2edaaf20-eb17-11f0-93ba-c64476ca29a9", "2f2efe9a-eb17-11f0-93ba-c64476ca29a9", "2f873c04-eb17-11f0-93ba-c64476ca29a9", "30328532-eb17-11f0-93ba-c64476ca29a9", "308945fc-eb17-11f0-93ba-c64476ca29a9", "30dfb86a-eb17-11f0-93ba-c64476ca29a9", "3137cfdc-eb17-11f0-93ba-c64476ca29a9", "318ea8ca-eb17-11f0-93ba-c64476ca29a9", "31e4e1e0-eb17-11f0-93ba-c64476ca29a9", "323b67ea-eb17-11f0-93ba-c64476ca29a9", "32e71c20-eb17-11f0-93ba-c64476ca29a9", "333d6c2e-eb17-11f0-93ba-c64476ca29a9", "33954c00-eb17-11f0-93ba-c64476ca29a9", "33ea91f6-eb17-11f0-93ba-c64476ca29a9", "34978708-eb17-11f0-93ba-c64476ca29a9", "34ee40fc-eb17-11f0-93ba-c64476ca29a9", "35f19170-eb17-11f0-93ba-c64476ca29a9", "3648aef6-eb17-11f0-93ba-c64476ca29a9", "369e2034-eb17-11f0-93ba-c64476ca29a9", "4717bb46-eb17-11f0-93ba-c64476ca29a9", "476b60c0-eb17-11f0-93ba-c64476ca29a9", "47c24552-eb17-11f0-93ba-c64476ca29a9", "49c8fdd2-eb17-11f0-93ba-c64476ca29a9", "4a202062-eb17-11f0-93ba-c64476ca29a9", "4a761d96-eb17-11f0-93ba-c64476ca29a9", "4b22aca0-eb17-11f0-93ba-c64476ca29a9", "4b7a4758-eb17-11f0-93ba-c64476ca29a9", "4bd011d8-eb17-11f0-93ba-c64476ca29a9", "4c273120-eb17-11f0-93ba-c64476ca29a9", "4c7d470e-eb17-11f0-93ba-c64476ca29a9", "4cd3f0cc-eb17-11f0-93ba-c64476ca29a9", "4d2a28e8-eb17-11f0-93ba-c64476ca29a9", "4d810708-eb17-11f0-93ba-c64476ca29a9", "4dd77caa-eb17-11f0-93ba-c64476ca29a9", "4e842216-eb17-11f0-93ba-c64476ca29a9", "4edb56ee-eb17-11f0-93ba-c64476ca29a9", "4f308a10-eb17-11f0-93ba-c64476ca29a9", "4f88b7b2-eb17-11f0-93ba-c64476ca29a9", "4fde6a18-eb17-11f0-93ba-c64476ca29a9", "5035b138-eb17-11f0-93ba-c64476ca29a9", "508bdf18-eb17-11f0-93ba-c64476ca29a9", "5fd29d32-eb16-11f0-93ba-c64476ca29a9", "608dbee6-eb16-11f0-93ba-c64476ca29a9", "60e406ca-eb16-11f0-93ba-c64476ca29a9", "619bf3b6-eb16-11f0-93ba-c64476ca29a9", "624a4632-eb16-11f0-93ba-c64476ca29a9", "62a05c34-eb16-11f0-93ba-c64476ca29a9", "62f67a06-eb16-11f0-93ba-c64476ca29a9", "63f82fb2-eb16-11f0-93ba-c64476ca29a9", "644c3710-eb16-11f0-93ba-c64476ca29a9", "64a24380-eb16-11f0-93ba-c64476ca29a9", "64f965e8-eb16-11f0-93ba-c64476ca29a9", "654f8d1a-eb16-11f0-93ba-c64476ca29a9", "65a5b190-eb16-11f0-93ba-c64476ca29a9", "65fc70de-eb16-11f0-93ba-c64476ca29a9", "6b302374-eb17-11f0-93ba-c64476ca29a9", "6b898702-eb17-11f0-93ba-c64476ca29a9", "6be282d0-eb17-11f0-93ba-c64476ca29a9", "6c36b13e-eb17-11f0-93ba-c64476ca29a9", "6ce3be38-eb17-11f0-93ba-c64476ca29a9", "6d3a59aa-eb17-11f0-93ba-c64476ca29a9", "6de84538-eb17-11f0-93ba-c64476ca29a9", "6e3e6c74-eb17-11f0-93ba-c64476ca29a9", "6e9512cc-eb17-11f0-93ba-c64476ca29a9", "6eeb308a-eb17-11f0-93ba-c64476ca29a9", "6f97e2c6-eb17-11f0-93ba-c64476ca29a9", "709a6dce-eb17-11f0-93ba-c64476ca29a9", "70f242a6-eb17-11f0-93ba-c64476ca29a9", "71476d80-eb17-11f0-93ba-c64476ca29a9", "719f025c-eb17-11f0-93ba-c64476ca29a9", "724b4a26-eb17-11f0-93ba-c64476ca29a9", "72f941da-eb17-11f0-93ba-c64476ca29a9", "7350227a-eb17-11f0-93ba-c64476ca29a9", "73a63f34-eb17-11f0-93ba-c64476ca29a9", "74529838-eb17-11f0-93ba-c64476ca29a9", "74a9fde4-eb17-11f0-93ba-c64476ca29a9", "7500695e-eb17-11f0-93ba-c64476ca29a9", "779b3856-eb17-11f0-93ba-c64476ca29a9", "77f0387e-eb17-11f0-93ba-c64476ca29a9", "7846f5ec-eb17-11f0-93ba-c64476ca29a9", "794b745e-eb17-11f0-93ba-c64476ca29a9", "7a4f1b58-eb17-11f0-93ba-c64476ca29a9", "7aa41478-eb17-11f0-93ba-c64476ca29a9", "7afc04d0-eb17-11f0-93ba-c64476ca29a9", "7bfef96e-eb17-11f0-93ba-c64476ca29a9", "7c56c40a-eb17-11f0-93ba-c64476ca29a9", "7cad190e-eb17-11f0-93ba-c64476ca29a9", "7d58664c-eb17-11f0-93ba-c64476ca29a9", "7db0d5b6-eb17-11f0-93ba-c64476ca29a9", "7e06aee6-eb17-11f0-93ba-c64476ca29a9", "7eb29d0a-eb17-11f0-93ba-c64476ca29a9", "7f60b368-eb17-11f0-93ba-c64476ca29a9", "7fb639f0-eb17-11f0-93ba-c64476ca29a9", "805e0aa4-eb17-11f0-93ba-c64476ca29a9", "80b59fa8-eb17-11f0-93ba-c64476ca29a9", "810accf8-eb17-11f0-93ba-c64476ca29a9", "8162a950-eb17-11f0-93ba-c64476ca29a9", "8a49e3fc-eb18-11f0-93ba-c64476ca29a9", "8b14ccfc-eb18-11f0-93ba-c64476ca29a9", "8b6b28b8-eb18-11f0-93ba-c64476ca29a9", "8c171c90-eb18-11f0-93ba-c64476ca29a9", "8c6d5b3c-eb18-11f0-93ba-c64476ca29a9", "8cb7a616-eb17-11f0-93ba-c64476ca29a9", "8cc568c2-eb18-11f0-93ba-c64476ca29a9", "8d0d89f0-eb17-11f0-93ba-c64476ca29a9", "8d1b2ce4-eb18-11f0-93ba-c64476ca29a9", "8d65289a-eb17-11f0-93ba-c64476ca29a9", "8d70f368-eb18-11f0-93ba-c64476ca29a9", "8dbb3546-eb17-11f0-93ba-c64476ca29a9", "8dc75d2a-eb18-11f0-93ba-c64476ca29a9", "8e1e90c2-eb18-11f0-93ba-c64476ca29a9", "8e62f06a-eb17-11f0-93ba-c64476ca29a9", "8e74ada4-eb18-11f0-93ba-c64476ca29a9", "8eb8eef2-eb17-11f0-93ba-c64476ca29a9", "8ecb3f16-eb18-11f0-93ba-c64476ca29a9", "8f0e7f84-eb17-11f0-93ba-c64476ca29a9", "8f23390a-eb18-11f0-93ba-c64476ca29a9", "8f61676c-eb17-11f0-93ba-c64476ca29a9", "8f798170-eb18-11f0-93ba-c64476ca29a9", "8fcf5da2-eb18-11f0-93ba-c64476ca29a9", "900d8006-eb17-11f0-93ba-c64476ca29a9", "9026950e-eb18-11f0-93ba-c64476ca29a9", "90d3af28-eb18-11f0-93ba-c64476ca29a9", "9110e2f4-eb17-11f0-93ba-c64476ca29a9", "912a9810-eb18-11f0-93ba-c64476ca29a9", "9167dac8-eb17-11f0-93ba-c64476ca29a9", "91805d04-eb18-11f0-93ba-c64476ca29a9", "91bd5a8e-eb17-11f0-93ba-c64476ca29a9", "91d5d1c6-eb18-11f0-93ba-c64476ca29a9", "9215c76e-eb17-11f0-93ba-c64476ca29a9", "922c3dfe-eb18-11f0-93ba-c64476ca29a9", "9282db82-eb18-11f0-93ba-c64476ca29a9", "92c219e2-eb17-11f0-93ba-c64476ca29a9", "92d91bbe-eb18-11f0-93ba-c64476ca29a9", "93187d78-eb17-11f0-93ba-c64476ca29a9", "933177d2-eb18-11f0-93ba-c64476ca29a9", "93703b6c-eb17-11f0-93ba-c64476ca29a9", "9387f864-eb18-11f0-93ba-c64476ca29a9", "93c5b786-eb17-11f0-93ba-c64476ca29a9", "93dd96ac-eb18-11f0-93ba-c64476ca29a9", "941ccdbe-eb17-11f0-93ba-c64476ca29a9", "9434df98-eb18-11f0-93ba-c64476ca29a9", "94730378-eb17-11f0-93ba-c64476ca29a9", "95cd2b86-eb17-11f0-93ba-c64476ca29a9", "9623f6d2-eb17-11f0-93ba-c64476ca29a9", "9679d854-eb17-11f0-93ba-c64476ca29a9", "a23e1088-eb17-11f0-93ba-c64476ca29a9", "a2937f1e-eb17-11f0-93ba-c64476ca29a9", "a2e6a9d2-eb17-11f0-93ba-c64476ca29a9", "a33ba8c4-eb17-11f0-93ba-c64476ca29a9", "a39416ee-eb17-11f0-93ba-c64476ca29a9", "a3e8716c-eb17-11f0-93ba-c64476ca29a9", "a440c150-eb17-11f0-93ba-c64476ca29a9", "a4958988-eb17-11f0-93ba-c64476ca29a9", "a4ec86ac-eb17-11f0-93ba-c64476ca29a9", "a599c0b0-eb17-11f0-93ba-c64476ca29a9", "a5efc1ea-eb17-11f0-93ba-c64476ca29a9", "ace1b21a-eb17-11f0-93ba-c64476ca29a9", "ad8e5da8-eb17-11f0-93ba-c64476ca29a9", "ae3b5dc8-eb17-11f0-93ba-c64476ca29a9", "ae91c398-eb17-11f0-93ba-c64476ca29a9", "af3eece4-eb17-11f0-93ba-c64476ca29a9", "af953bf8-eb17-11f0-93ba-c64476ca29a9", "afeb7126-eb17-11f0-93ba-c64476ca29a9", "b042b698-eb17-11f0-93ba-c64476ca29a9", "b0981480-eb17-11f0-93ba-c64476ca29a9", "b0eee292-eb17-11f0-93ba-c64476ca29a9", "b145210c-eb17-11f0-93ba-c64476ca29a9", "b19c3e2e-eb17-11f0-93ba-c64476"]
test_labels = ["2dd6eca6-eb17-11f0-93ba-c64476ca29a9","2e8357de-eb17-11f0-93ba-c64476ca29a9","34410248-eb17-11f0-93ba-c64476ca29a9","354559c8-eb17-11f0-93ba-c64476ca29a9","48176e88-eb17-11f0-93ba-c64476ca29a9","4e2e0b56-eb17-11f0-93ba-c64476ca29a9","50e20938-eb17-11f0-93ba-c64476ca29a9","61f2bb7e-eb16-11f0-93ba-c64476ca29a9","634dbfa0-eb16-11f0-93ba-c64476ca29a9","6c8d3ce8-eb17-11f0-93ba-c64476ca29a9","6f421292-eb17-11f0-93ba-c64476ca29a9","7045498e-eb17-11f0-93ba-c64476ca29a9","72a24ec0-eb17-11f0-93ba-c64476ca29a9","78f5070e-eb17-11f0-93ba-c64476ca29a9","79f8417a-eb17-11f0-93ba-c64476ca29a9","7b5246a6-eb17-11f0-93ba-c64476ca29a9","7d03b6c4-eb17-11f0-93ba-c64476ca29a9","7f0926b6-eb17-11f0-93ba-c64476ca29a9","8ab2ceb2-eb18-11f0-93ba-c64476ca29a9","8fb7f726-eb17-11f0-93ba-c64476ca29a9","90ba15fa-eb17-11f0-93ba-c64476ca29a9","9576deac-eb17-11f0-93ba-c64476ca29a9","a1e8c57e-eb17-11f0-93ba-c64476ca29a9","a542fe60-eb17-11f0-93ba-c64476ca29a9","a6463c64-eb17-11f0-93ba-c64476ca29a9","ade4864c-eb17-11f0-93ba-c64476ca29a9","b2f6a41c-eb17-11f0-93ba-c64476ca29a9","b4fdd2d0-eb17-11f0-93ba-c64476ca29a9","ba1445ec-eb17-11f0-93ba-c64476ca29a9","bb165340-eb17-11f0-93ba-c64476ca29a9","bb6e9df2-eb17-11f0-93ba-c64476ca29a9","bc7055f6-eb17-11f0-93ba-c64476ca29a9","bd1e3842-eb17-11f0-93ba-c64476ca29a9","bd74318e-eb17-11f0-93ba-c64476ca29a9","dbba2a96-eb16-11f0-93ba-c64476ca29a9","de427f7a-eb16-11f0-93ba-c64476ca29a9","de9544da-eb16-11f0-93ba-c64476ca29a9","dfef42fe-eb16-11f0-93ba-c64476ca29a9","e3a58dc2-eb16-11f0-93ba-c64476ca29a9","e6095f62-eb16-11f0-93ba-c64476ca29a9","e6b56c08-eb16-11f0-93ba-c64476ca29a9","e7b8c6f4-eb16-11f0-93ba-c64476ca29a9","eabef95e-eb16-11f0-93ba-c64476ca29a9","ec198486-eb16-11f0-93ba-c64476ca29a9","ed1cd7c0-eb16-11f0-93ba-c64476ca29a9","ed72cfe0-eb16-11f0-93ba-c64476ca29a9","edc8bda6-eb16-11f0-93ba-c64476ca29a9","f17b7bdc-eb16-11f0-93ba-c64476ca29a9","f1e0ad08-eb17-11f0-93ba-c64476ca29a9","f28d7b14-eb17-11f0-93ba-c64476ca29a9","f4eb0854-eb17-11f0-93ba-c64476ca29a9","f597d6c4-eb17-11f0-93ba-c64476ca29a9","f748030e-eb17-11f0-93ba-c64476ca29a9","f8a26b22-eb17-11f0-93ba-c64476ca29a9","f8f96ff8-eb17-11f0-93ba-c64476ca29a9","f9a624d2-eb17-11f0-93ba-c64476ca29a9","fa5375ba-eb17-11f0-93ba-c64476ca29a9","ff747120-eb17-11f0-93ba-c64476ca29a9"]
validation_labels = ["001a711a-eb18-11f0-93ba-c64476ca29a9","2fdcd02e-eb17-11f0-93ba-c64476ca29a9","329177b6-eb17-11f0-93ba-c64476ca29a9","359af54a-eb17-11f0-93ba-c64476ca29a9","486fd3ca-eb17-11f0-93ba-c64476ca29a9","48c5e062-eb17-11f0-93ba-c64476ca29a9","491c5956-eb17-11f0-93ba-c64476ca29a9","4973332a-eb17-11f0-93ba-c64476ca29a9","4acc54ea-eb17-11f0-93ba-c64476ca29a9","60328cec-eb16-11f0-93ba-c64476ca29a9","613aaba6-eb16-11f0-93ba-c64476ca29a9","63a19f94-eb16-11f0-93ba-c64476ca29a9","6651a2ca-eb16-11f0-93ba-c64476ca29a9","6d8ff518-eb17-11f0-93ba-c64476ca29a9","6fef4b56-eb17-11f0-93ba-c64476ca29a9","71f58230-eb17-11f0-93ba-c64476ca29a9","73fd69bc-eb17-11f0-93ba-c64476ca29a9","789f2e38-eb17-11f0-93ba-c64476ca29a9","79a25dc8-eb17-11f0-93ba-c64476ca29a9","7ba98722-eb17-11f0-93ba-c64476ca29a9","7e5c6b9c-eb17-11f0-93ba-c64476ca29a9","80093d3a-eb17-11f0-93ba-c64476ca29a9","8bc179b6-eb18-11f0-93ba-c64476ca29a9","8e1003b4-eb17-11f0-93ba-c64476ca29a9","9063475c-eb17-11f0-93ba-c64476ca29a9","907ccb2c-eb18-11f0-93ba-c64476ca29a9","926b4bf8-eb17-11f0-93ba-c64476ca29a9","94c9dd92-eb17-11f0-93ba-c64476ca29a9","952056e0-eb17-11f0-93ba-c64476ca29a9","ad366292-eb17-11f0-93ba-c64476ca29a9","aee82648-eb17-11f0-93ba-c64476ca29a9","b34cdf9e-eb17-11f0-93ba-c64476ca29a9","b3f96fac-eb17-11f0-93ba-c64476ca29a9","b657ab74-eb17-11f0-93ba-c64476ca29a9","b8b985ae-eb17-11f0-93ba-c64476ca29a9","b90f0178-eb17-11f0-93ba-c64476ca29a9","b9659614-eb17-11f0-93ba-c64476ca29a9","bbc47f4c-eb17-11f0-93ba-c64476ca29a9","bc19ccb8-eb17-11f0-93ba-c64476ca29a9","bcc6ea92-eb17-11f0-93ba-c64476ca29a9","e09c5200-eb16-11f0-93ba-c64476ca29a9","e0f2124e-eb16-11f0-93ba-c64476ca29a9","e1481572-eb16-11f0-93ba-c64476ca29a9","e4ff9a1e-eb16-11f0-93ba-c64476ca29a9","e70ce4b0-eb16-11f0-93ba-c64476ca29a9","e762279a-eb16-11f0-93ba-c64476ca29a9","e80b2be2-eb16-11f0-93ba-c64476ca29a9","eb15bc62-eb16-11f0-93ba-c64476ca29a9","ee1f8988-eb16-11f0-93ba-c64476ca29a9","eeccb6d0-eb16-11f0-93ba-c64476ca29a9","f1253e34-eb16-11f0-93ba-c64476ca29a9","f37f490e-eb16-11f0-93ba-c64476ca29a9","f391611a-eb17-11f0-93ba-c64476ca29a9","f480c4ea-eb16-11f0-93ba-c64476ca29a9","f79f2922-eb17-11f0-93ba-c64476ca29a9","faa90aa2-eb17-11f0-93ba-c64476ca29a9","fe70d908-eb17-11f0-93ba-c64476ca29a9"]
total_labels = [lb for lb in os.listdir('/Applications/Deep Learning/Face Detection/data/labels')]


''' 
Need to take care of the Except Cases
'''

#Moving labels into corresponding Dataset sub-directory
for name in train_labels:
    try:
        shutil.copy(os.path.join(label_path, name + '.json'), os.path.join(base_path, "train_set/labels"))
    except:
        print(name)
        # print('File not found: ', FileNotFoundError())

# print(len(os.listdir('/Applications/Deep Learning/Face Detection/data/train_set/labels')))
# print(len(os.listdir('/Applications/Deep Learning/Face Detection/data/train_set/images')))


# for name in validation_labels:
#     try:
#         shutil.copy(os.path.join(label_path, name + '.json'), os.path.join(base_path, "validation_set/labels"))
#     except: #Havent Considered this yet :(
#         print(name)


# for name in test_labels:
#     try:
#         shutil.copy(os.path.join(label_path, name + '.json'), os.path.join(base_path, "test_set/labels"))
#     except:
#         print(name)

30dfb86a-eb17-11f0-93ba-c64476ca29a9
508bdf18-eb17-11f0-93ba-c64476ca29a9
624a4632-eb16-11f0-93ba-c64476ca29a9
63f82fb2-eb16-11f0-93ba-c64476ca29a9
644c3710-eb16-11f0-93ba-c64476ca29a9
64f965e8-eb16-11f0-93ba-c64476ca29a9
654f8d1a-eb16-11f0-93ba-c64476ca29a9
65a5b190-eb16-11f0-93ba-c64476ca29a9
6e9512cc-eb17-11f0-93ba-c64476ca29a9
8dc75d2a-eb18-11f0-93ba-c64476ca29a9
8ecb3f16-eb18-11f0-93ba-c64476ca29a9
8f23390a-eb18-11f0-93ba-c64476ca29a9
8f61676c-eb17-11f0-93ba-c64476ca29a9
91bd5a8e-eb17-11f0-93ba-c64476ca29a9
91d5d1c6-eb18-11f0-93ba-c64476ca29a9
922c3dfe-eb18-11f0-93ba-c64476ca29a9
93dd96ac-eb18-11f0-93ba-c64476ca29a9
b19c3e2e-eb17-11f0-93ba-c64476


In [ ]:
label_path = "/Applications/Deep Learning/Face Detection/data/labels"

splits = {
    "train_set": train_labels,
    "validation_set": validation_labels,
    "test_set": test_labels,
}

missing = []

for split, labels in splits.items():
    dst = os.path.join(base_path, split, "labels")
    os.makedirs(dst, exist_ok=True)

    for name in labels:
        src = os.path.join(label_path, name + ".json")
        if os.path.exists(src):
            shutil.copy(src, dst)
        else:
            missing.append(name)

print(f"Missing label files: {len(missing)}")


### Applying Image Augmentation on Images and Labels using Albumentations

In [15]:
augmentor = alb.Compose([alb.RandomCrop(width=900, height=900), #How big Augmented Picture will be
                        alb.HorizontalFlip(p=0.5), #
                        alb.RandomBrightnessContrast(p=0.2),
                        alb.RandomGamma(p=0.2), 
                        alb.RGBShift(p=0.2), 
                        alb.VerticalFlip(p=0.5)],
                        bbox_params = alb.BboxParams(format='albumentations', label_fields=['class_labels']
                        )
                    )


In [16]:
cv2.imread('/Applications/Deep Learning/Face Detection/data/images/00c7b1fe-eb18-11f0-93ba-c64476ca29a9.jpg').shape

2.25, 3

(2.25, 3)

In [237]:
#Load a particuar Image to test
train_url = "/Applications/Deep Learning/Face Detection/data/train_set"
img_example = cv2.imread(
    os.path.join(train_url,"images", "00c7b1fe-eb18-11f0-93ba-c64476ca29a9.jpg")
)


with open(
    '/Applications/Deep Learning/Face Detection/data/train_set/labels/00c7b1fe-eb18-11f0-93ba-c64476ca29a9.json',
    "r"
) as f:
    label = json.load(f)

#Class of Label
print('Class: ', label['shapes'][0]['label'])
print(label['shapes'][0]['points'][1][0])

Class:  Face
1462.5641025641025


In [238]:
h, w = img_example.shape[:2]  # get height and width

coordinates = [0, 0, 0, 0]
coordinates[0] = label['shapes'][0]['points'][0][0]
coordinates[1] = label['shapes'][0]['points'][0][1]
coordinates[2] = label['shapes'][0]['points'][1][0]
coordinates[3] = label['shapes'][0]['points'][1][1]

coordinates = list(np.divide(coordinates, [w,h,w,h]))

print('New Coordinates', coordinates)


New Coordinates [0.30235042735042744, 0.14624881291547953, 0.7617521367521367, 0.8964862298195632]


In [ ]:
# Apply Augmentation and View Results 
augmented = augmentor(image=img_example, bboxes=[coordinates], class_labels=['face'])
print(augmented['image'].shape)


# Draw rectangle using normalized coordinates scaled to actual image size
cv2.rectangle(
    augmented['image'],
    tuple((np.array(augmented['bboxes'][0][:2]) * [h,h]).astype(int)),   # top-left
    tuple((np.array(augmented['bboxes'][0][2:]) * [h,h]).astype(int)),   # bottom-right
    (255, 0, 0), 2
)

# Show result
plt.imshow(cv2.cvtColor(augmented['image'], cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


### Build and Run Augmentation Pipeline

In [ ]:
'''
Fix our Augmentation cus right now its jibberish
'''


for partition in ['train_set', 'validation_set', 'test_set']:
    for image in os.listdir(os.path.join(base_path, partition, 'images')):
        img = cv2.imread(os.path.join(base_path, partition, 'images', image))
        coordinates = [0,0,0.00001,0.00001]

        #Add Json to name of image
        label_path = os.path.join(base_path, partition, 'labels', f'{image.split(".")[0]}.json')

        #These are the pictures that have been identified as a face
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)
        
            coordinates[0] = label['shapes'][0]['points'][0][0]
            coordinates[1] = label['shapes'][0]['points'][0][1]
            coordinates[2] = label['shapes'][0]['points'][1][0]
            coordinates[3] = label['shapes'][0]['points'][1][1]
            h,w = img.shape[:2]
            coordinates = list(np.divide(coordinates, [w,h,w,h]))

        #Create Annotation for Missing Image Labels and Create 120 Augmented Images
    
        for x in range(120):
            try:
                augmented = augmentor(image=img, bboxes=[coordinates], class_labels=['Face'])
                cv2.imwrite(os.path.join('/Applications/Deep Learning/Face Detection/aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])
                annotation = {}
                annotation['image'] = f'{image.split(".")[0]}.{x}.jpg'

                #Image has a label for Face(Classifying Class Labels)
                if os.path.exists(label_path):
                    if (len(augmented['bboxes']) == 0):
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0
                    else:
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                #That Image Originally did not have label
                else:
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 #Not a Face

                #Write these labels
                with open(os.path.join('/Applications/Deep Learning/Face Detection/aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

            except Exception as e:
                print(e)



### Load Augmented Images to TensorFlow Dataset

In [4]:
train_images = tf.data.Dataset.list_files('/Applications/Deep Learning/Face Detection/aug_data/train_set/images/*.jpg', shuffle = False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (120,120)))
train_images = train_images.map(lambda x: x/255)

print(len(train_images))


31920


In [5]:
test_images = tf.data.Dataset.list_files('/Applications/Deep Learning/Face Detection/aug_data/test_set/images/*.jpg', shuffle = False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (120,120)))
test_images = test_images.map(lambda x: x/255)

In [6]:
val_images = tf.data.Dataset.list_files('/Applications/Deep Learning/Face Detection/aug_data/validation_set/images/*.jpg', shuffle = False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (120,120))) #Compress Image
val_images = val_images.map(lambda x: x/255) #Scale Image --> Sigmoid Function as all values between [0,1]

### Prepare Labels

In [7]:
#Build Label Loading Function
def load_labels(label_path):
    #Return  the Class in Bounding Box and the Class
    with open(label_path.numpy(), 'r', encoding='utf-8') as f:
        label = json.load(f)
    return [label['class']], label['bbox']

In [8]:
def tf_load_labels(x):
    cls, bbox = tf.py_function(
        load_labels,
        [x],
        [tf.uint8, tf.float16]
    )

    # Explicitly set shapes
    cls.set_shape([1])     
    bbox.set_shape([4])   # xmin, ymin, xmax, ymax

    return cls, bbox


In [9]:
#Load Labels to Tensorflow Dataset
training_labels = tf.data.Dataset.list_files('/Applications/Deep Learning/Face Detection/aug_data/train_set/labels/*.json', shuffle=False)

print('Value Passed to Load Label Function:', training_labels.as_numpy_iterator().next()) #Gets Full URL and is passed into load_label function to get corresponding values

#Return The Train Label Class and Bounding Box in corresponding format
# training_labels = training_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16])) --> tf.py_function caused unknown shape in Tensor
training_labels = training_labels.map(
    tf_load_labels,
    num_parallel_calls=tf.data.AUTOTUNE
)

test_labels = tf.data.Dataset.list_files('/Applications/Deep Learning/Face Detection/aug_data/test_set/labels/*.json', shuffle = False)
# test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))
test_labels = test_labels.map(
    tf_load_labels,
    num_parallel_calls=tf.data.AUTOTUNE
)
val_labels = tf.data.Dataset.list_files('/Applications/Deep Learning/Face Detection/aug_data/validation_set/labels/*.json', shuffle = False)
# val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))
val_labels = val_labels.map(
    tf_load_labels,
    num_parallel_calls=tf.data.AUTOTUNE
)

print(f'Output of Label Function for Train --> \nClass:{training_labels.as_numpy_iterator().next()[0]} \nBounding Box: {training_labels.as_numpy_iterator().next()[1]}')
#tf.py_function allows wrap any Python function into a TF Data pipeline

Value Passed to Load Label Function: b'/Applications/Deep Learning/Face Detection/aug_data/train_set/labels/00717294-eb18-11f0-93ba-c64476ca29a9.0.json'
Output of Label Function for Train --> 
Class:[1] 
Bounding Box: [0.5493 0.407  1.     0.994 ]


### Combine Label and Image Samples

In [31]:
#Create Final Dataset

train = tf.data.Dataset.zip((train_images, training_labels)) #Combine together 
train = train.shuffle(2046) #2046 not len(train_images) too slow
train = train.batch(10000) #Each batch will be 100 Images and 100 Labels
train = train.prefetch(4) #Gets images beforehand so when training on certain batch already preprocessing the next


print(training_labels)


test = tf.data.Dataset.zip((test_images, test_labels)) 
test = test.shuffle(2046)
test = test.batch(10000) 
test = test.prefetch(4) 



val = tf.data.Dataset.zip((val_images, val_labels)) #Combine together 
val = train.shuffle(2046)
val  = val.batch(10000) #Each batch will be 100 Images and 100 Labels
val = val.prefetch(4) #Gets images beforehand so when training on certain batch already preprocessing the next

#train.as_numpy_iterator().next()[0].shape()

<_ParallelMapDataset element_spec=(TensorSpec(shape=(1,), dtype=tf.uint8, name=None), TensorSpec(shape=(4,), dtype=tf.float16, name=None))>


In [ ]:
#View Images and Annotations
data_samples = train.as_numpy_iterator()
next_batch = data_samples.next()

fig, ax = plt.subplots(ncols=5, nrows=20, figsize=(20,10))

for idx in range(5):
    sample_image = next_batch[0][idx]
    sample_coordinates = next_batch[1][1][idx]

    cv2.rectangle(sample_image, tuple(np.multiply(sample_coordinates[:2], [120,120]).astype(int),
    tuple(np.multiply(sample_coordinates[2:], [120,120])).astype(int)))

    ax[idx].imshow(sample_image)



### Building Deep Learning NN using Functional API

1) Classification problem: Identify if its face or not
2) Regression: Calculating the Bounding box

https://www.geeksforgeeks.org/computer-vision/vgg-16-cnn-model/

In [32]:
#Create Base NN 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten,GlobalMaxPooling2D, Dropout, ReLU
from tensorflow.keras.applications import VGG16 #VGG16 Architecture, Pretrained used for Image Classifications

In [33]:
#Download VGG16(Classification Model but remove the final activation functions)
vgg = VGG16(include_top=False, weights = None)
vgg.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print('Info on the Dimension of Boundary:',  train.as_numpy_iterator().next()[1])
print('Info regarding Labels:', train.as_numpy_iterator().next()[0])

In [34]:
#Create Custom VG16 Model

def new_model():
    #Input Dimensions
    input_layer = Input(shape=(120,120,3)) 

    #Pass input_layer as input to Architecture(No Pretrained Weights due to Network issues)
    vgg = VGG16(include_top=False, weights=None)(input_layer) 

    #Classification Problem --> Classify whether Face or Not (Use Sigmoid Activation )
    f1 = GlobalMaxPooling2D()(vgg) #Condese all information of the vgg16 and returning max values and will give back 512 values
    class1 = Dense(2048, activation='relu')(f1) #Pass F1 and ReLU activation function takes max value and returns 2048 outputs
    class2 = Dense(1, activation='sigmoid')(class1) #1 Output value

    #Regression Problem(Solve the Optimal Weights to find the coordinates of BBox)
    f2 = GlobalMaxPooling2D()(vgg)
    regression1 = Dense(2048, activation='relu')(f2)
    regression2 = Dense(4, activation='sigmoid')(regression1) #Regress 2 takes 4 Output for each corner of box 


    face_identifier = Model(inputs=input_layer, outputs = [class2, regression2])
    return face_identifier



#### Testing out the new VGG16 NN Architecture

In [35]:
face_identifier = new_model()
face_identifier.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 120, 120,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vgg16 (Functional)  │ (None, 3, 3, 512) │ 14,714,688 │ input_layer_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 512)       │          0 │ vgg16[0][0]       │
│ (GlobalMaxPooling2… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 512)       │          0 │ vgg16[0][0]       │
│ (GlobalMaxPooling2… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 2048)      │  1,050,624 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 2048)      │  1,050,624 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 1)         │      2,049 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 4)         │      8,196 │ dense_10[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 16,826,181 (64.19 MB)

 Trainable params: 16,826,181 (64.19 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
#Take an Example Sample from Pipeline
X,y = train.as_numpy_iterator().next()
print('Image Shape:', X.shape)

Image Shape: (100, 120, 120, 3)


In [ ]:
#Predicting without Optimization(Need to Fix Prediction Rate: Augmentation Images gone wrong)
class_type, predicted_coordinates = face_identifier.predict(X)
print('Class Predicted', class_type)
print('Bounding Box Predicted Coordinates:', predicted_coordinates)

### Define Losses and Optimizers to Evaluate Model

* Localization Loss: https://stats.stackexchange.com/questions/319243/object-detection-loss-function-yolo

In [36]:
#Define LR
batches_per_epoch = len(train)
lr_decay =(1./0.75-1)/batches_per_epoch
print('How much LR Drops for each epoch: ', lr_decay)

#Define Optimizer(Adam Optimizer)
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001, decay = lr_decay)

How much LR Drops for each epoch:  0.0010416666666666664


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [37]:
#Create Localization Loss for Bbox and Classification Loss (Binary Class Entropy)

def localization_loss(y_true, y_pred):
    #Distance between Actual and Predicated Coordinate
    delta_coordinate = tf.reduce_sum(tf.square(y_true[:,:2] - y_pred[:,:2]))

    #ACTUAL HEIGHT AND WDITH OF BOX (REMEMBER HEIGHT STORED IN [3] AND [1] WHILE WIDTH IS [0] AND [2])
    h_true = y_true[:,3] - y_true[:,1]
    w_true = y_true[:,2] - y_true[:,0]

    #Predicted Height and Wdith of BBox
    h_pred = y_pred[:,3] - y_pred[:,1]
    w_pred = y_pred[:,2] - y_pred[:,0]

    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true - h_pred))

    #Localization Loss
    return delta_coordinate + delta_size

In [38]:
classification_loss = tf.keras.losses.BinaryCrossentropy()
loc_loss = localization_loss #Another way to call our Regression Loss

### Test out Loss Metrics

In [ ]:
#Convert Tensor back to Numpy value
print('Testing Sample Localization Loss: ', localization_loss(y[1], predicted_coordinates).numpy()) #y[1] holds our BBox dimensions
print('Testing Sample Classification Loss: ', classification_loss(y[0], class_type).numpy()) #y[0] holds the class label


### Functions to Train and Test Model

In [39]:
''' 
Our Model Class for the Neural Network and Training it 
Uses Super Inheritance
'''
class FaceModel(Model):
    def __init__(self, pre_built_model, **kwargs):
        super().__init__(**kwargs) #Super Inheritance
        self.model = pre_built_model
    
    #Used to COnfigure the Model that we will be Training but has not started training yet
    def compile(self, optimizer, classification_loss, localization_loss, **kwargs):
        super().compile(**kwargs)
        self.classloss = classification_loss
        self.loc_loss = localization_loss
        self.optimizer = optimizer

    #Training the NN and passed the batches of images for it to learn
    def train_step(self, batch, **kwargs):
        X, y = batch #Split between Images and Labels
        # print(y[1])

        #Compute all math operations, calculate gradient after forward pass to then do Back propogation
        with tf.GradientTape() as tape:
            #Pass Images to the Model and it will output classes and BBox for each Image
            classes, coordinates = self.model(X, training=True)
            batch_classloss = self.classloss(y[0], classes)
            batch_locloss = self.loc_loss(tf.cast(y[1], tf.float32), coordinates)

            #Can change the 0.5 to a different value if their an anomaly
            total_loss = batch_locloss + 0.5 * batch_classloss

            #Calculate Gradient --> How to improve the model (Negative Direction is path to Minimize and Magnitude is how much to update Parameter Weights)
            gradient = tape.gradient(total_loss, self.model.trainable_variables)

        #How our model applys gradient to improve the optimization of the parameter weights
        self.optimizer.apply_gradients(zip(gradient, self.model.trainable_variables))

        #Store Losses in a Dictionary to access in future
        return {'Total loss': total_loss, 'classification_loss':batch_classloss, 'localization_loss': batch_locloss}


    #Testing the Model once Training is Complete. How to ensure Model is not Overfitting
    def test_step(self, batch, **kwargs):
        X, y = batch
        classes, coordinates= self.model(X, training = False)

        batch_classloss = self.classloss(y[0], classes)
        batch_locloss = self.loc_loss(tf.cast(y[1], tf.float32), coordinates)
        total_loss = batch_locloss + 0.5 * batch_classloss

        #Training over should not need gradients to help train weights, gives final losses for Images in Batch
        return {'Total loss': total_loss, 'classification_loss':batch_classloss, 'localization_loss': batch_locloss}


    #Necessary if want to use the built in function .predict() 
    def call(self, X, **kwargs):
        return self.model(X, **kwargs, training = False)




In [40]:
#New NN Model after Implementing all functions 
face_model = FaceModel(face_identifier)

face_model.compile(optimizer, classification_loss, loc_loss)


### Training the NN (Saving Logs into a Logs Directory)

In [41]:
log_directory = 'logs'
#Save all Logs to directory to view for reference and to graph 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_directory)


#Training model on n batches if we use train.take(n)
hist = face_model.fit(train, epochs = 10, validation_data = val, callbacks = [tensorboard_callback])

320
Epoch 1/40
 13/320 ━━━━━━━━━━━━━━━━━━━━ 3:22:18 40s/step - Total loss: 74.6390 - classification_loss: 0.6925 - localization_loss: 74.2927

### Plot Performance

In [ ]:
hist.history

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20,5))

ax[0].plot(hist.history['total_loss'], color='red', label='loss')
ax[0].plot(hist.history['val_total_loss'], color='blue', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='red', label='class loss')
ax[1].plot(hist.history['val_class_loss'], color='blue', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='red', label='regress loss')
ax[2].plot(hist.history['val_regress_loss'], color='blue', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()

### Make Prediction on Test Set of Images

In [ ]:
test_data = test.as_numpy_iterator()
next_test_sample = test_data.next()
y_pred = face_model.predict(test_sample[0])


In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = test_sample[0][idx]
    sample_coords = y_pred[1][idx]
    
    if yhat[0][idx] > 0.9:
        cv2.rectangle(sample_image, 
                      tuple(np.multiply(sample_coords[:2], [120,120]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [120,120]).astype(int)), 
                            (255,0,0), 2)
    
    ax[idx].imshow(sample_image)

### Save Model for Future Purposes

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
face_model.save('model.h5')
model = load_model('model.h5')